In [ ]:
import pandas as pd
from pathlib import Path
import json
import argparse

# # At the top of your script
# parser = argparse.ArgumentParser()
# parser.add_argument('--digest-id', type=str, help="Filter by digest_id prefix (e.g. 20250621T16)")
# args = parser.parse_args()



# === PATHS ===
PFOUT_DIR = Path("./data/pf_out")
ARTICLE_OUT_PATH = Path("./data/article_quotes/articles_exploded.jsonl")
IDEA_OUT_PATH = Path("./data/idea_cluster/seed_ideas_exploded.jsonl")

# Crear directorios de salida si no existen
ARTICLE_OUT_PATH.parent.mkdir(parents=True, exist_ok=True)
IDEA_OUT_PATH.parent.mkdir(parents=True, exist_ok=True)


# /home/matias/Documents/media_monitor/data/master_ref.csv
MASTER_REF_PATH = "./data/master_ref.csv"
master_ref = pd.read_csv(MASTER_REF_PATH)

master_ref = master_ref.loc[master_ref.index_id.str.len() == 10]

# master_ref["article_key"] = master_ref["digest_file"] + "::" + master_ref["article_id"].astype(str)
# rss_index = master_ref.set_index("article_key").to_dict(orient="index")
rss_index = master_ref.set_index("key").to_dict(orient="index")

# === Inicializar listas acumuladoras ===
article_rows = []
seed_idea_rows = []

# # === Cargar todos los archivos pfout ===
# if args.digest_id:
#     pf_files = list(PFOUT_DIR.glob(f"pfout_{args.digest_id}_*.jsonl"))
# else:
    
    
pf_files = list(PFOUT_DIR.glob("pfout_*.jsonl"))


print(f"🔍 Encontrados {len(pf_files)} archivos para procesar...")

for pf_file in pf_files:
    df = pd.read_json(pf_file, lines=True)
    
    for _, row in df.iterrows():
        # --- Parseo robusto desde digest_group_id ---
        digest_group_id = row.get("digest_group_id", "")
        parts = digest_group_id.split("::")
        if len(parts) >= 2:
            digest_ts = parts[0]              # Ej: '20250619T1600'
            window_type = parts[1]            # Ej: '8h_window'
            digest_file = f"{window_type}_{digest_ts}"  # Ej: '8h_window_20250619T1600'
        else:
            # Fallback si digest_group_id está ausente o mal formado
            digest_file = str(row.get("id_digest"))
            window_type = str(row.get("window_type"))

        # --- Metadatos base ---
        line_metadata = {
            "line_number": row.get("line_number"),
            "id_digest": digest_file,   # ← valor correctamente formateado
            "window_type": window_type,
            "topic": row.get("topic"),
            "group_number": row.get("group_number"),
        }

        # === Artículos ===
        clusters = row.get("clustered_agenda_table", {}).get("clustered_agenda_table", [])
        for cluster in clusters:
            topic = cluster["topic"]
            article_ids = cluster["article_ids"]
            titles = cluster["deduplicated_titles"]
            for aid, title in zip(article_ids, titles):
                article_key = f"{digest_file}::{aid}"

                article_rows.append({
                    **line_metadata,
                    "cluster_topic": topic,
                    "article_id": aid,
                    "title": title,
                    "source_file": pf_file.name,
                }) #index_id,uid,Topic,Title,Published,Source,Link

        # === Ideas Semilla ===
        seed_ideas = row.get("seed_ideas", {}).get("seed_ideas", [])
        for idea in seed_ideas:
            seed_idea_rows.append({
                **line_metadata,
                **idea,
                "source_file": pf_file.name
            })

# === Guardar resultados incrementales ===
new_articles_df = pd.DataFrame(article_rows)
new_ideas_df = pd.DataFrame(seed_idea_rows)

def concat_existing(output_path: Path, new_df: pd.DataFrame, subset_cols=None):
    if output_path.exists():
        old_df = pd.read_json(output_path, lines=True)
        if subset_cols:
            combined = pd.concat([old_df, new_df], ignore_index=True).drop_duplicates(subset=subset_cols)
        else:
            combined = pd.concat([old_df, new_df], ignore_index=True)
    else:
        combined = new_df
    return combined

combined_articles = concat_existing(ARTICLE_OUT_PATH, new_articles_df, subset_cols=["article_id", "title", "source_file"])
combined_ideas = concat_existing(IDEA_OUT_PATH, new_ideas_df, subset_cols=["idea_id", "idea_title", "source_file"])


combined_articles.to_json(ARTICLE_OUT_PATH, orient="records", lines=True, force_ascii=False)
combined_ideas.to_json(IDEA_OUT_PATH, orient="records", lines=True, force_ascii=False)

print(f"✅ Guardados:")
print(f"  - Artículos → {ARTICLE_OUT_PATH}")
print(f"  - Ideas     → {IDEA_OUT_PATH}")


# === Enriquecer con master_ref ===
MASTER_REF_PATH = Path("./data/master_ref.csv")
SCRAPED_PATH = Path("./data/scraped_links.jsonl")
ENRICHED_OUT_PATH = Path("./data/article_quotes/articles_to_scrape.jsonl")

# === Enriquecer artículos combinados con master_ref y scraped_links ===
full_article_df = pd.read_json(ARTICLE_OUT_PATH, lines=True, dtype={"id_digest": str, "article_id": str})

# --- Extraer 'hour' y 'article_key' ---
full_article_df['hour'] = full_article_df['source_file'].apply(lambda x: x.split('_')[1] + '00')

def extract_digest_file(source_file):
    # pfout_20250611T04_010127.jsonl → 4h_window_20250611T0400
    parts = source_file.replace(".jsonl", "").split("_")
    window = parts[1]
    hour = parts[2][:11]  # Toma 20250611T04
    return f"{window}00"

# window_type
full_article_df["digest_file"] = full_article_df["window_type"] +'_'+ full_article_df["source_file"].apply(extract_digest_file)
full_article_df['key'] = full_article_df['digest_file'] + "::" + full_article_df['article_id'].astype(str)

# master_ref = master_ref.loc[master_ref.index_id.str.len() == 10]

# full_article_df['article_key'] = full_article_df['id_digest'].astype(str) + "::" + full_article_df['article_id'].astype(str)


# === Merge con master_ref.csv ===
if MASTER_REF_PATH.exists():
    print("🔗 Enriqueciendo con master_ref.csv...")
    master_ref = pd.read_csv(MASTER_REF_PATH)
    master_ref = master_ref.loc[master_ref.index_id.str.len() == 10]
    
    master_ref['article_key'] = master_ref['digest_file'] + "::" + master_ref['article_id'].astype(str)

    master_subset = master_ref[[
        'key', 'article_key', 'index_id', 'Source', 'Title', 'Published', 'Link'
    ]].drop_duplicates()

    merged = full_article_df.merge(master_subset, how='left', on='key')
else:
    print("⚠️ master_ref.csv no encontrado, salteando enriquecimiento.")
    merged = full_article_df.copy()


# === Merge con scraped_links.jsonl ===
if SCRAPED_PATH.exists():
    print("🔗 Agregando contenido scrapeado...")
    content_df = pd.read_json(SCRAPED_PATH, lines=True)[['index_id', 'scraped_data']]

    if 'index_id' in merged.columns:
        merged['index_id'] = merged['index_id'].astype(str)
        content_df['index_id'] = content_df['index_id'].astype(str)
        merged = merged.merge(content_df, how='left', on='index_id')
    else:
        print("⚠️ 'index_id' no está presente en merged — no se puede hacer el merge con contenido scrapeado.")
        merged['scraped_data'] = None
else:
    print("⚠️ scraped_links.jsonl no encontrado, salteando merge con contenido.")
    merged['scraped_data'] = None

merged



🔍 Encontrados 0 archivos para procesar...
✅ Guardados:
  - Artículos → data/article_quotes/articles_exploded.jsonl
  - Ideas     → data/idea_cluster/seed_ideas_exploded.jsonl
🔗 Enriqueciendo con master_ref.csv...


,line_number,id_digest,window_type,topic,group_number,cluster_topic,article_id,title,source_file,hour,digest_file,key,article_key,index_id,Source,Title,Published,Link
0,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,3,Condena a CFK: gremios K llaman a la calle y l...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::3,4h_window_20250611T0400::3,72be5a5bbe,El Eco,Condena a CFK: gremios K llaman a la calle y l...,2025-06-10 20:06:18+00:00,https://news.google.com/rss/articles/CBMiqAFBV...
1,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,26,Cristina Kirchner condenada: gremios anunciaro...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::26,4h_window_20250611T0400::26,c2d7de48a5,El Destape,Cristina Kirchner condenada: gremios anunciaro...,2025-06-10 20:27:58+00:00,https://news.google.com/rss/articles/CBMi0gFBV...
2,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,29,Gremios cortan los accesos a CABA ante el posi...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::29,4h_window_20250611T0400::29,16d2d8155b,InfoRegión,Gremios cortan los accesos a CABA ante el posi...,2025-06-10 20:28:02+00:00,https://news.google.com/rss/articles/CBMixgFBV...
3,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,84,"Gremios cercanos a CFK harán huelgas, pero las...",pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::84,4h_window_20250611T0400::84,aed240b190,La Nación,Dólar hoy: la reacción del mercado al combo de...,2025-06-10 22:15:00+00:00,https://news.google.com/rss/articles/CBMizgFBV...
4,0,0,4h_window,Actividad_y_Empleo,1,Reclamos salariales y condiciones laborales en...,37,Paro de 48 horas en el Hospital Garrahan: recl...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::37,4h_window_20250611T0400::37,0bc9c507ba,La Noticia Web,Paro de 48 horas en el Hospital Garrahan: recl...,2025-06-10 20:40:54+00:00,https://news.google.com/rss/articles/CBMi2gFBV...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,68,Dólar: las proyecciones de economistas y consu...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::68,NaN,NaN,NaN,NaN,NaN,NaN
2733,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,80,Artana cuestionó a Cristina Kirchner por su cr...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::80,NaN,NaN,NaN,NaN,NaN,NaN
2734,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,88,Coremberg: corrupción en causa Cuadernos afect...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::88,NaN,NaN,NaN,NaN,NaN,NaN
2735,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,91,"Dólar, inflación y elecciones: las proyeccione...",pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::91,NaN,NaN,NaN,NaN,NaN,NaN


In [132]:
# import re
# filename = "headlines_4h_window_20250621T20_Sector_Externo_01.md"
# # filename = "./data/output_digests/headlines_4h_window_20250621T20_Sector_Externo_01.md"
# re.match(r"^headlines_(\w+_window)_(\d{8}T\d{2})_(.+)_(\d{2})\.md$", filename)

<re.Match object; span=(0, 52), match='headlines_4h_window_20250621T20_Sector_Externo_01>

🔗 Agregando contenido scrapeado...


In [127]:
merged

,line_number,id_digest,window_type,topic,group_number,cluster_topic,article_id,title,source_file,hour,digest_file,key,article_key,index_id,Source,Title,Published,Link,scraped_data
0,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,3,Condena a CFK: gremios K llaman a la calle y l...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::3,4h_window_20250611T0400::3,72be5a5bbe,El Eco,Condena a CFK: gremios K llaman a la calle y l...,2025-06-10 20:06:18+00:00,https://news.google.com/rss/articles/CBMiqAFBV...,Ads\n\n\nBUSCAR...\n\nNEWSLETTERS\nSUSCRIBITE\...
1,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,26,Cristina Kirchner condenada: gremios anunciaro...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::26,4h_window_20250611T0400::26,c2d7de48a5,El Destape,Cristina Kirchner condenada: gremios anunciaro...,2025-06-10 20:27:58+00:00,https://news.google.com/rss/articles/CBMi0gFBV...,\n\nEl Destape\nEN VIVO\n\nSindicatos\nTodos l...
2,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,29,Gremios cortan los accesos a CABA ante el posi...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::29,4h_window_20250611T0400::29,16d2d8155b,InfoRegión,Gremios cortan los accesos a CABA ante el posi...,2025-06-10 20:28:02+00:00,https://news.google.com/rss/articles/CBMixgFBV...,"miércoles, junio 11 2025\nRegión 90.5\nArchivo..."
3,0,0,4h_window,Actividad_y_Empleo,1,Protestas y huelgas relacionadas con la conden...,84,"Gremios cercanos a CFK harán huelgas, pero las...",pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::84,4h_window_20250611T0400::84,aed240b190,La Nación,Dólar hoy: la reacción del mercado al combo de...,2025-06-10 22:15:00+00:00,https://news.google.com/rss/articles/CBMizgFBV...,SECCIONES\n\n\nINICIAR SESIÓN\n\nSuscribite po...
4,0,0,4h_window,Actividad_y_Empleo,1,Reclamos salariales y condiciones laborales en...,37,Paro de 48 horas en el Hospital Garrahan: recl...,pfout_20250611T04_010127.jsonl,20250611T0400,4h_window_20250611T0400,4h_window_20250611T0400::37,4h_window_20250611T0400::37,0bc9c507ba,La Noticia Web,Paro de 48 horas en el Hospital Garrahan: recl...,2025-06-10 20:40:54+00:00,https://news.google.com/rss/articles/CBMi2gFBV...,"jueves 12 de junio, 2025\n\nBuscador\nInicio\n..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2736,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,68,Dólar: las proyecciones de economistas y consu...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::68,NaN,nan,NaN,NaN,NaN,NaN,NaN
2737,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,80,Artana cuestionó a Cristina Kirchner por su cr...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::80,NaN,nan,NaN,NaN,NaN,NaN,NaN
2738,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,88,Coremberg: corrupción en causa Cuadernos afect...,pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::88,NaN,nan,NaN,NaN,NaN,NaN,NaN
2739,11,8h_window_20250622T0000,8h_window,Tipo_de_Cambio_y_Reservas,2,Tipo de Cambio y Reservas,91,"Dólar, inflación y elecciones: las proyeccione...",pfout_20250622T00_010601.jsonl,20250622T0000,8h_window_20250622T0000,8h_window_20250622T0000::91,NaN,nan,NaN,NaN,NaN,NaN,NaN


In [ ]:
merged